In [1]:
import sys
from pathlib import Path
import qlib
import pandas as pd
from qlib.config import REG_CN
from qlib.contrib.model.gbdt import LGBModel
from qlib.contrib.data.handler import Alpha360
from qlib.utils import init_instance_by_config
from qlib.contrib.strategy import TopkDropoutStrategy
from qlib.contrib.report import analysis_model, analysis_position
# from qlib.contrib.evaluate import (
#     backtest as normal_backtest,
#     risk_analysis,
# )
from qlib.utils import exists_qlib_data, init_instance_by_config, flatten_dict
from qlib.workflow import R
from qlib.workflow.record_temp import SignalRecord, PortAnaRecord
from qlib.utils import flatten_dict
from qlib.data.dataset.loader import QlibDataLoader
from qlib.contrib.data.handler import Alpha158   #Alpha158内置指标体系
from qlib.data.dataset.loader import QlibDataLoader
import qlib
from qlib.contrib.data.handler import Alpha158   #Alpha158内置指标体系


provider_uri = "./qlib_data/cn_data"  # 原始行情数据存放目录
qlib.init(provider_uri=provider_uri, region=REG_CN)  # 初始化
market = "csi300"
benchmark = "SH000300"

#数据处理器参数配置
data_handler_config = {
    "start_time": "2008-01-01",
    "end_time": "2020-08-01",
    "fit_start_time": "2008-01-01",
    "fit_end_time": "2014-12-31",
    # "start_time": "2020-01-01",
    # "end_time": "2020-02-21",
    # "fit_start_time": "2020-01-01",  # 模型跑数据的开始时间
    # "fit_end_time": "2020-01-31",
    "instruments": market,
    "infer_processors" : [
                                    {'class': 'FilterCol',
                                     'kwargs': {'fields_group': 'feature', 
                                                'col_list': ["RESI5", "WVMA5", "RSQR5", "KLEN", "RSQR10", "CORR5", "CORD5", "CORR10", 
                            "ROC60", "RESI10", "VSTD5", "RSQR60", "CORR60", "WVMA60", "STD5", 
                            "RSQR20", "CORD60", "CORD10", "CORR20", "KLOW"
                        ]
                                     }},
                                    {'class': 'RobustZScoreNorm', # RobustZScoreNorm和Fillna，用于进行标准化和填充缺失值。
                                     'kwargs': {'fields_group': 'feature', 
                                                'clip_outlier': True}},
                                    {'class': 'Fillna', 
                                     'kwargs': {'fields_group': 'feature'}}],
    "learn_processors": [{'class': 'DropnaLabel'}, #DropnaLabel删除标注中含有缺失值的样本。

                                    # 对预测的目标进行截面排序处理  DropnaLabel 和 CSRankNorm 预处理器，用于对标签进行处理。
                                    {'class': 'CSRankNorm', 'kwargs': {'fields_group': 'label'}}],

                                    # 预测的目标
                                    'label': ["Ref($close, -2) / Ref($close, -1) - 1"] ,#下一日收益率, Ref($close, -1)表示下一日收盘价
                                    
}
    
# 任务参数配置
task = {
    "model": {  # 模型参数配置
        # 模型类
        "class": "TransformerModel",
        # 模型类所在模块
        "module_path": "qlib.contrib.model.pytorch_transformer_ts_cw",
        "kwargs": {  # 模型超参数配置
            'seed': 0,
            'n_jobs': 20,
            'GPU':2
        }, 
    },
    "dataset": {  # 　因子库数据集参数配置
        # 数据集类，是Dataset with Data(H)andler的缩写，即带数据处理器的数据集
        "class": "TSDatasetH",
        # 数据集类所在模块
        "module_path": "qlib.data.dataset",
        "kwargs": {  # 数据集参数配置
            "handler": {  # 数据集使用的数据处理器配置
                #"class": "Alpha158",  # 数据处理器类，继承自DataHandlerLP
                "module_path": "qlib.contrib.data.handler",  # 数据处理器类所在模块
                "class": "Alpha158",
                "kwargs": data_handler_config,  # 数据处理器参数配置
            },
            "segments": {  # 数据集划分标准
                "train": ("2008-01-01", "2014-12-31"), # 此时段的数据为训练集
                "valid": ("2015-01-01", "2016-12-31"), # 此时段的数据为验证集
                "test": ("2017-01-01", "2020-08-01"),  # 此时段的数据为测试集
                # "train": ("2020-01-01", "2020-01-31"),  # 此时段的数据为训练集
                # "valid": ("2020-01-31", "2020-02-20"),  # 此时段的数据为验证集
                # "test": ("2020-02-20", "2020-02-21"),  # 此时段的数据为测试集
            },
        },
    },

}

# 实例化模型对象
model = init_instance_by_config(task["model"])

# 实例化因子库数据集，从基础行情数据计算出的包含所有特征（因子）和标签值的数据集。
dataset = init_instance_by_config(task["dataset"])  # DatasetH

ModuleNotFoundError. CatBoostModel are skipped. (optional: maybe installing CatBoostModel can fix it.)
ModuleNotFoundError. XGBModel is skipped(optional: maybe installing xgboost can fix it).


/home/shared/anaconda3/envs/seg/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[25296:MainThread](2023-06-21 13:20:38,627) INFO - qlib.Initialization - [config.py:413] - default_conf: client.
[25296:MainThread](2023-06-21 13:20:38,631) INFO - qlib.workflow - [expm.py:31] - experiment manager uri is at file:/home/shared/qlib-main/mlruns
[25296:MainThread](2023-06-21 13:20:38,633) INFO - qlib.Initialization - [__init__.py:74] - qlib successfully initialized based on client settings.
[25296:MainThread](2023-06-21 13:20:38,634) INFO - qlib.Initialization - [__init__.py:76] - data_path={'__DEFAULT_FREQ': PosixPath('/home/shared/qlib-main/qlib_data/cn_data')}
[25296:MainThread](2023-06-21 13:20:38,865) INFO - qlib.TransformerModel - [pytorch_transformer_ts_cw.py:77] - Naive Transformer:
batch_size : 4096
devi

In [2]:
# start exp to train model
with R.start(experiment_name="train_model"):
    R.log_params(**flatten_dict(task))
    model.fit(dataset)
    R.save_objects(trained_model=model)
    rid = R.get_recorder().id

[25296:MainThread](2023-06-21 13:21:39,022) INFO - qlib.workflow - [expm.py:316] - <mlflow.tracking.client.MlflowClient object at 0x7feb8ca55220>
[25296:MainThread](2023-06-21 13:21:39,033) INFO - qlib.workflow - [exp.py:260] - Experiment 1 starts running ...
[25296:MainThread](2023-06-21 13:21:39,537) INFO - qlib.workflow - [recorder.py:339] - Recorder fa899b865999493fb797f54b577c98b7 starts running under Experiment 1 ...
Not a git repository
To compare two paths outside a working tree:
usage: git diff [--no-index] <path> <path>
[25296:MainThread](2023-06-21 13:21:39,670) INFO - qlib.workflow - [recorder.py:372] - Fail to log the uncommitted code of $CWD when run `git diff`
fatal: 不是一个 git 仓库（或者直至挂载点 / 的任何父目录）
停止在文件系统边界（未设置 GIT_DISCOVERY_ACROSS_FILESYSTEM）。
[25296:MainThread](2023-06-21 13:21:39,813) INFO - qlib.workflow - [recorder.py:372] - Fail to log the uncommitted code of $CWD when run `git status`
Not a git repository
To compare two paths outside a working tree:
usage: git diff

[25296:MainThread](2023-06-22 06:39:33,766) INFO - qlib.TransformerModel - [pytorch_transformer_ts_cw.py:292] - evaluating...
[25296:MainThread](2023-06-22 06:40:17,237) INFO - qlib.TransformerModel - [pytorch_transformer_ts_cw.py:295] - train -0.998177, valid -0.999857
[25296:MainThread](2023-06-22 06:40:17,244) INFO - qlib.TransformerModel - [pytorch_transformer_ts_cw.py:289] - Epoch2:
[25296:MainThread](2023-06-22 06:40:17,246) INFO - qlib.TransformerModel - [pytorch_transformer_ts_cw.py:290] - training...


[25296:MainThread](2023-06-22 12:28:14,470) INFO - qlib.TransformerModel - [pytorch_transformer_ts_cw.py:292] - evaluating...
[25296:MainThread](2023-06-22 12:28:59,147) INFO - qlib.TransformerModel - [pytorch_transformer_ts_cw.py:295] - train -0.991719, valid -0.994818
[25296:MainThread](2023-06-22 12:28:59,156) INFO - qlib.TransformerModel - [pytorch_transformer_ts_cw.py:289] - Epoch3:
[25296:MainThread](2023-06-22 12:28:59,158) INFO - qlib.TransformerModel - [pytorch_transformer_ts_cw.py:290] - training...


[25296:MainThread](2023-06-22 18:16:49,460) INFO - qlib.TransformerModel - [pytorch_transformer_ts_cw.py:292] - evaluating...
[25296:MainThread](2023-06-22 18:17:32,704) INFO - qlib.TransformerModel - [pytorch_transformer_ts_cw.py:295] - train -0.991857, valid -0.995878
[25296:MainThread](2023-06-22 18:17:32,712) INFO - qlib.TransformerModel - [pytorch_transformer_ts_cw.py:289] - Epoch4:
[25296:MainThread](2023-06-22 18:17:32,715) INFO - qlib.TransformerModel - [pytorch_transformer_ts_cw.py:290] - training...


[25296:MainThread](2023-06-23 00:07:30,127) INFO - qlib.TransformerModel - [pytorch_transformer_ts_cw.py:292] - evaluating...
[25296:MainThread](2023-06-23 00:08:13,813) INFO - qlib.TransformerModel - [pytorch_transformer_ts_cw.py:295] - train -0.997573, valid -1.001482
[25296:MainThread](2023-06-23 00:08:13,817) INFO - qlib.TransformerModel - [pytorch_transformer_ts_cw.py:289] - Epoch5:
[25296:MainThread](2023-06-23 00:08:13,818) INFO - qlib.TransformerModel - [pytorch_transformer_ts_cw.py:290] - training...


[25296:MainThread](2023-06-23 06:03:26,704) INFO - qlib.TransformerModel - [pytorch_transformer_ts_cw.py:292] - evaluating...
[25296:MainThread](2023-06-23 06:04:10,259) INFO - qlib.TransformerModel - [pytorch_transformer_ts_cw.py:295] - train -0.991359, valid -0.996485
[25296:MainThread](2023-06-23 06:04:10,263) INFO - qlib.TransformerModel - [pytorch_transformer_ts_cw.py:289] - Epoch6:
[25296:MainThread](2023-06-23 06:04:10,265) INFO - qlib.TransformerModel - [pytorch_transformer_ts_cw.py:290] - training...


[25296:MainThread](2023-06-23 12:05:54,537) INFO - qlib.TransformerModel - [pytorch_transformer_ts_cw.py:292] - evaluating...
[25296:MainThread](2023-06-23 12:06:37,887) INFO - qlib.TransformerModel - [pytorch_transformer_ts_cw.py:295] - train -1.002068, valid -1.005956
[25296:MainThread](2023-06-23 12:06:37,892) INFO - qlib.TransformerModel - [pytorch_transformer_ts_cw.py:289] - Epoch7:
[25296:MainThread](2023-06-23 12:06:37,893) INFO - qlib.TransformerModel - [pytorch_transformer_ts_cw.py:290] - training...


[25296:MainThread](2023-06-24 11:20:45,400) INFO - qlib.TransformerModel - [pytorch_transformer_ts_cw.py:292] - evaluating...
[25296:MainThread](2023-06-24 11:21:28,587) INFO - qlib.TransformerModel - [pytorch_transformer_ts_cw.py:295] - train -0.991997, valid -0.996173
[25296:MainThread](2023-06-24 11:21:28,589) INFO - qlib.TransformerModel - [pytorch_transformer_ts_cw.py:307] - early stop
[25296:MainThread](2023-06-24 11:21:28,590) INFO - qlib.TransformerModel - [pytorch_transformer_ts_cw.py:317] - best score: -0.994818 @ 2
[25296:MainThread](2023-06-24 11:21:28,907) INFO - qlib.timer - [log.py:117] - Time cost: 0.000s | waiting `async_log` Done


In [3]:
###################################
# prediction, backtest & analysis
###################################
port_analysis_config = {
    "executor": {
        "class": "SimulatorExecutor",
        "module_path": "qlib.backtest.executor",
        "kwargs": {
            "time_per_step": "day",
            "generate_portfolio_metrics": True,
        },
    },
    "strategy": {
        "class": "TopkDropoutStrategy",
        "module_path": "qlib.contrib.strategy",
        "kwargs": {
            "model": model,
            "dataset": dataset,
            "topk": 50,
            "n_drop": 5,
        },
    },
    "backtest": {
        "start_time": "2017-01-01",
        "end_time": "2020-08-01",
        "account": 100000000,
        "benchmark": benchmark,
        "exchange_kwargs": {
            "freq": "day",
            "limit_threshold": 0.095,
            "deal_price": "close",
            "open_cost": 0.0005,
            "close_cost": 0.0015,
            "min_cost": 5,
        },
    },
}

# backtest and analysis
with R.start(experiment_name="backtest_analysis"):
    recorder = R.get_recorder(recorder_id=rid, experiment_name="train_model")
    model = recorder.load_object("trained_model")

    # prediction
    recorder = R.get_recorder()
    ba_rid = recorder.id
    sr = SignalRecord(model, dataset, recorder)
    sr.generate()

    # backtest & analysis
    par = PortAnaRecord(recorder, port_analysis_config, "day")
    par.generate()

[25296:MainThread](2023-06-24 11:21:28,949) INFO - qlib.workflow - [expm.py:316] - <mlflow.tracking.client.MlflowClient object at 0x7fea203d6d90>
[25296:MainThread](2023-06-24 11:21:28,958) INFO - qlib.workflow - [exp.py:260] - Experiment 2 starts running ...
[25296:MainThread](2023-06-24 11:21:28,984) INFO - qlib.workflow - [recorder.py:339] - Recorder f7dacf9d4f7e4f6f827b4594d337eef3 starts running under Experiment 2 ...
Not a git repository
To compare two paths outside a working tree:
usage: git diff [--no-index] <path> <path>
[25296:MainThread](2023-06-24 11:21:29,166) INFO - qlib.workflow - [recorder.py:372] - Fail to log the uncommitted code of $CWD when run `git diff`
fatal: 不是一个 git 仓库（或者直至挂载点 / 的任何父目录）
停止在文件系统边界（未设置 GIT_DISCOVERY_ACROSS_FILESYSTEM）。
[25296:MainThread](2023-06-24 11:21:29,337) INFO - qlib.workflow - [recorder.py:372] - Fail to log the uncommitted code of $CWD when run `git status`
Not a git repository
To compare two paths outside a working tree:
usage: git diff

'The following are prediction results of the TransformerModel model.'
                          score
datetime   instrument          
2017-01-03 SH600000   -0.018606
           SH600008    0.043327
           SH600009    0.097937
           SH600010    0.057394
           SH600015    0.024099


[25296:MainThread](2023-06-24 11:21:37,703) INFO - qlib.backtest caller - [__init__.py:94] - Create new exchange
[25296:MainThread](2023-06-24 11:21:45,313) WARNING - qlib.online operator - [exchange.py:216] - factor.day.bin file not exists or factor contains `nan`. Order using adjusted_price.
[25296:MainThread](2023-06-24 11:21:45,315) WARNING - qlib.online operator - [exchange.py:218] - trade unit 100 is not supported in adjusted_price mode.
[25296:MainThread](2023-06-24 11:21:55,136) WARNING - qlib.data - [data.py:662] - load calendar error: freq=day, future=True; return current calendar!
[25296:MainThread](2023-06-24 11:21:55,139) WARNING - qlib.data - [data.py:665] - You can get future calendar by referring to the following document: https://github.com/microsoft/qlib/blob/main/scripts/data_collector/contrib/README.md
[25296:MainThread](2023-06-24 11:21:55,160) WARNING - qlib.BaseExecutor - [executor.py:121] - `common_infra` is not set for <qlib.backtest.executor.SimulatorExecutor 

'The following are analysis results of benchmark return(1day).'
                       risk
mean               0.000477
std                0.012295
annualized_return  0.113561
information_ratio  0.598699
max_drawdown      -0.370479
'The following are analysis results of the excess return without cost(1day).'
                       risk
mean               0.000348
std                0.004981
annualized_return  0.082848
information_ratio  1.078201
max_drawdown      -0.101280
'The following are analysis results of the excess return with cost(1day).'
                       risk
mean               0.000168
std                0.004980
annualized_return  0.039908
information_ratio  0.519487
max_drawdown      -0.109592
'The following are analysis results of indicators(1day).'
     value
ffr    1.0
pa     0.0
pos    0.0


In [4]:
from qlib.contrib.report import analysis_model, analysis_position
from qlib.data import D
recorder = R.get_recorder(recorder_id=ba_rid, experiment_name="backtest_analysis")
print(recorder)
pred_df = recorder.load_object("pred.pkl")
pred_df_dates = pred_df.index.get_level_values(level='datetime') #提取pred_df中的日期信息，将日期数据提取到pred_df_dates变量中。
report_normal_df = recorder.load_object("portfolio_analysis/report_normal_1day.pkl")#加载了保存的标准报告（portfolio report）数据，将其赋值给report_normal_df变量。
positions = recorder.load_object("portfolio_analysis/positions_normal_1day.pkl")#加载了保存的仓位数据，将其赋值给positions变量。
analysis_df = recorder.load_object("portfolio_analysis/port_analysis_1day.pkl")#加载了保存的组合分析数据，将其赋值给analysis_df变量。

{'class': 'Recorder', 'id': 'f7dacf9d4f7e4f6f827b4594d337eef3', 'name': 'mlflow_recorder', 'experiment_id': '2', 'start_time': '2023-06-24 11:21:28', 'end_time': '2023-06-24 11:22:12', 'status': 'FINISHED'}


In [5]:
analysis_df

risk
excess_return_without_cost mean               0.000348
                           std                0.004981
                           annualized_return  0.082848
                           information_ratio  1.078201
                           max_drawdown      -0.101280
excess_return_with_cost    mean               0.000168
                           std                0.004980
                           annualized_return  0.039908
                           information_ratio  0.519487
                           max_drawdown      -0.109592

In [6]:
analysis_df

risk
excess_return_without_cost mean               0.000348
                           std                0.004981
                           annualized_return  0.082848
                           information_ratio  1.078201
                           max_drawdown      -0.101280
excess_return_with_cost    mean               0.000168
                           std                0.004980
                           annualized_return  0.039908
                           information_ratio  0.519487
                           max_drawdown      -0.109592

In [9]:
from qlib.workflow.record_temp import SignalRecord, SigAnaRecord, PortAnaRecord

record_config = [
    {
        "class": SignalRecord,
        "module_path": "qlib.workflow.record_temp",
        "kwargs": {
            "model": "<MODEL>",
            "dataset": "<DATASET>"
        }
    },
    {
        "class": SigAnaRecord,
        "module_path": "qlib.workflow.record_temp",
        "kwargs": {
            "ana_long_short": False,
            "ann_scaler": 252
        }
    },
    {
        "class": PortAnaRecord,
        "module_path": "qlib.workflow.record_temp",
        "kwargs": {
            "config": "*port_analysis_config"
        }
    }
]
record_config


[{'class': qlib.workflow.record_temp.SignalRecord,
  'module_path': 'qlib.workflow.record_temp',
  'kwargs': {'model': '<MODEL>', 'dataset': '<DATASET>'}},
 {'class': qlib.workflow.record_temp.SigAnaRecord,
  'module_path': 'qlib.workflow.record_temp',
  'kwargs': {'ana_long_short': False, 'ann_scaler': 252}},
 {'class': qlib.workflow.record_temp.PortAnaRecord,
  'module_path': 'qlib.workflow.record_temp',
  'kwargs': {'config': '*port_analysis_config'}}]

: 